# *ETL Proccess*

This notebook processes transaction data from a CSV file for the xpate test task. The goal is to clean, transform, and save the data in a desired format.

---

## **1. Import Libraries**

We start by importing the necessary Python libraries for data manipulation and analysis.


In [12]:
import pandas as pd
import numpy as np
import ast

## **2. Load and Inspect the Data**

We load the CSV file and take a quick look at the initial structure of the data.


In [13]:
# Load the dataset
df = pd.read_csv("/Users/d.grisulonoks/Downloads/input_test_xpate.csv")

# Display the first few rows of the dataframe
df.head()


,type,entry,is_correction,created,network,net_settlement_currency,net_settlement_impact,settlement_service_code,settlement_service_name,settlement_service_id,type_specific,transaction_reference,transaction_type,gross_settlement_amount
0,secondPresentment-dms,1,False,11.07.2024 13:56,mastercard,EUR,credit,EU00000008,"EUR - Deutsche Bank, Frankfurt",EU00000008-EUR,"{ ""preDispute"": ""none""}",c90335ff-e59e-4957-bc6a-02fa62265b03,purchase,250.0
1,firstChargeback-dms,1,False,30.07.2024 13:54,mastercard,EUR,debit,EU00000008,"EUR - Deutsche Bank, Frankfurt",EU00000008-EUR,"{ ""preDispute"": ""collaborationRefund""}",af005b4d-d557-4c54-85b9-ca78cea6c038,purchase,100.0
2,firstChargeback-dms,1,False,2024-07-03 03:11:10.012,mastercard,EUR,debit,EU00000008,"EUR - Deutsche Bank, Frankfurt",EU00000008-EUR,"{ ""preDispute"": ""collaborationRefund""}",9715cab9-ebed-4c26-adf2-65974d397da6,purchase,20.0
3,firstChargeback-dms,1,False,03.07.2024 03:11,mastercard,EUR,debit,EU00000008,"EUR - Deutsche Bank, Frankfurt",EU00000008-EUR,"{ ""preDispute"": ""collaborationRefund""}",8d8d3629-55c7-4e27-be22-48edc3dc493f,purchase,20.0
4,firstChargeback-dms,1,False,04.07.2024 03:11,mastercard,EUR,debit,EU00000008,"EUR - Deutsche Bank, Frankfurt",EU00000008-EUR,"{ ""preDispute"": ""none""}",9576aa98-7de5-4a25-9979-259956da2b5d,purchase,100.0


## **3. Data Cleaning**

Here we clean the dataset by removing unnecessary columns and converting data types where needed.


In [14]:
# Drop unnecessary columns
df = df.drop(columns=['entry','created', 'is_correction', 'network', 'settlement_service_code','settlement_service_name','settlement_service_id'])

# Convert 'gross_settlement_amount' to integers
df['gross_settlement_amount'] = df['gross_settlement_amount'].astype(int)

# Display the cleaned DataFrame
df.head()


,type,net_settlement_currency,net_settlement_impact,type_specific,transaction_reference,transaction_type,gross_settlement_amount
0,secondPresentment-dms,EUR,credit,"{ ""preDispute"": ""none""}",c90335ff-e59e-4957-bc6a-02fa62265b03,purchase,250
1,firstChargeback-dms,EUR,debit,"{ ""preDispute"": ""collaborationRefund""}",af005b4d-d557-4c54-85b9-ca78cea6c038,purchase,100
2,firstChargeback-dms,EUR,debit,"{ ""preDispute"": ""collaborationRefund""}",9715cab9-ebed-4c26-adf2-65974d397da6,purchase,20
3,firstChargeback-dms,EUR,debit,"{ ""preDispute"": ""collaborationRefund""}",8d8d3629-55c7-4e27-be22-48edc3dc493f,purchase,20
4,firstChargeback-dms,EUR,debit,"{ ""preDispute"": ""none""}",9576aa98-7de5-4a25-9979-259956da2b5d,purchase,100


## **4. Creating Debit and Credit Columns**

We create new columns for debit and credit based on the `net_settlement_impact`.


In [15]:
# Create 'debit' and 'credit' columns
df['debit'] = np.where(df['net_settlement_impact'] == 'credit', df['gross_settlement_amount'], 0)
df['credit'] = np.where(df['net_settlement_impact'] == 'debit', df['gross_settlement_amount'], 0)

# Drop columns no longer needed
df = df.drop(columns=['gross_settlement_amount','net_settlement_impact'])

# Rename columns for clarity
df.rename(columns={'net_settlement_currency': 'currency'}, inplace=True)

# Display the updated DataFrame
df.head()


,type,currency,type_specific,transaction_reference,transaction_type,debit,credit
0,secondPresentment-dms,EUR,"{ ""preDispute"": ""none""}",c90335ff-e59e-4957-bc6a-02fa62265b03,purchase,250,0
1,firstChargeback-dms,EUR,"{ ""preDispute"": ""collaborationRefund""}",af005b4d-d557-4c54-85b9-ca78cea6c038,purchase,0,100
2,firstChargeback-dms,EUR,"{ ""preDispute"": ""collaborationRefund""}",9715cab9-ebed-4c26-adf2-65974d397da6,purchase,0,20
3,firstChargeback-dms,EUR,"{ ""preDispute"": ""collaborationRefund""}",8d8d3629-55c7-4e27-be22-48edc3dc493f,purchase,0,20
4,firstChargeback-dms,EUR,"{ ""preDispute"": ""none""}",9576aa98-7de5-4a25-9979-259956da2b5d,purchase,0,100


## **5. Creating and Transforming Additional Columns**

In this section, we derive additional columns such as `action_type`, `type_v2`, and `acct_num`.


In [16]:
# Create 'action_type' and extract 'preDispute' from 'type_specific'
df['action_type'] = df['type']
df['preDispute'] = df['type_specific'].apply(lambda x: ast.literal_eval(x).get('preDispute'))

# Create 'type_v2' based on 'preDispute'
df['type_v2'] = np.where(df['preDispute'] != 'none', df['preDispute'], df['type'])

# Drop unnecessary columns
df.drop(columns=['preDispute','type','type_specific'], inplace=True)

# Add 'acct_num' column with a constant value
df['acct_num'] = 2100

# Replace specific values in 'type_v2' with more descriptive names
df['type_v2'] = df['type_v2'].replace({
    'secondPresentment-dms': 'Second Presentment',
    'collaborationRefund': 'Collaboration Refund',
    'firstChargeback-dms': 'Chargeback',
    'preArbitration-dms': 'Arbitration'
})

# Rename 'type_v2' to 'type'
df.rename(columns={'type_v2': 'type'}, inplace=True)

# Display the transformed DataFrame
df.head()


,currency,transaction_reference,transaction_type,debit,credit,action_type,type,acct_num
0,EUR,c90335ff-e59e-4957-bc6a-02fa62265b03,purchase,250,0,secondPresentment-dms,Second Presentment,2100
1,EUR,af005b4d-d557-4c54-85b9-ca78cea6c038,purchase,0,100,firstChargeback-dms,Collaboration Refund,2100
2,EUR,9715cab9-ebed-4c26-adf2-65974d397da6,purchase,0,20,firstChargeback-dms,Collaboration Refund,2100
3,EUR,8d8d3629-55c7-4e27-be22-48edc3dc493f,purchase,0,20,firstChargeback-dms,Collaboration Refund,2100
4,EUR,9576aa98-7de5-4a25-9979-259956da2b5d,purchase,0,100,firstChargeback-dms,Chargeback,2100


## **6. Reordering and Saving the Final Data**

Finally, we reorder the columns to match the desired output format and save the DataFrame to a CSV file.


In [1]:
# Define the desired column order
order = ['type','acct_num','action_type','currency','transaction_type', 'transaction_reference','debit','credit']

# Reorder the DataFrame
df = df[order]

# Save the DataFrame to a CSV file
df.to_csv('/Users/d.grisulonoks/Downloads/output_xpate_v6.csv', index=False)

# Display the final DataFrame
df.head()


NameError: name 'df' is not defined

---

### **Conclusion**

This notebook successfully cleaned, transformed, and saved the transaction data as required. The final output CSV is ready for further analysis or reporting.
